# **패키지 설치 & api key 설정**

In [1]:
!pip install pytesseract PyPDF2
!pip install pdfplumber pdf2image
# Install Poppler utilities
!sudo apt-get update
!sudo apt-get install -y poppler-utils
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 48.7 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu22

In [2]:
!pip install openai
!pip install langchain
!pip install pypdf
!pip install faiss-cpu
!pip install tiktoken
!pip install -U langchain langchain-community langchain-core
!pip install langchain openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [3]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.4 MB/s eta 0:00:00


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
import os
import openai
from langchain.chat_models import ChatOpenAI

#  API 키 설정
OPENAI_API_KEY = "   "
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = openai.OpenAI(api_key=OPENAI_API_KEY)
openai.api_key = OPENAI_API_KEY

# **구글 드라이브 마운트**

In [64]:
from google.colab import drive
drive.mount('/content/drive')

import glob
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Streamlit**

In [65]:
%%writefile app.py

import streamlit as st
from PIL import Image
import tempfile
import os
import sys
import re
sys.path.append('/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/수능 국어 AI 튜터링 시스템/문학')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/수능 국어 AI 튜터링 시스템/비문학')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/수능 국어 AI 튜터링 시스템/화작')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/TAVE 프로젝트_STUBO/수능 국어 AI 튜터링 시스템/언매')

# 🔹 내부 파이프라인 함수 불러오기
from pipeline_literature import pipeline_literature, retriever_literature_answer, retriever_literature_recommend, tag_dict_literature # 문학
from pipeline_non_literature import extract_text_with_underlines, ask_gpt, tag_from_image, recommend_for_external_problem, attach_image_paths, pipeline_non_literature # 비문학
from pipeline_speechcomp import analyze_problem # 화법과 작문
from pipeline_langmedia import # 언어와 매체

# ✅ Streamlit 설정
st.set_page_config(page_title="STUBO", layout="centered")
st.title("📘 수능 국어 AI 튜터링 시스템")

st.markdown("""
📝 문학 지문 이미지와 문제 이미지를 업로드하면 자동으로
OCR → 정답/해설 생성 → 유사 문제 추천까지 수행합니다.
""")

# 🔹 과목 선택 (문학, 비문학, 화법과 작문, 언어와 매체)
subject = st.radio("📚 분석할 과목을 선택하세요", ["문학", "비문학", "화법과 작문", "언어와 매체"])

# 🔹 이미지 업로드
passage_image = st.file_uploader("1️⃣ 지문 이미지 업로드", type=["png", "jpg", "jpeg"], key="passage")
question_image = st.file_uploader("2️⃣ 문제 이미지 업로드", type=["png", "jpg", "jpeg"], key="question")

# 🔹 유사 문제 추천 개수 선택
top_k = st.slider("🔁 유사 문제 추천 개수", 1, 5, 2)

# 🔹 실행 버튼
if st.button("🚀 실행하기"):
    if not passage_image or not question_image:
        st.warning("⚠️ 지문과 문제 이미지를 모두 업로드해주세요.")
    else:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_passage:
            tmp_passage.write(passage_image.read())
            passage_img_path = tmp_passage.name

        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_question:
            tmp_question.write(question_image.read())
            question_img_path = tmp_question.name

        # 🔸 업로드 이미지 표시
        st.image(passage_img_path, caption="📖 업로드한 지문 이미지", use_container_width=True)
        st.image(question_img_path, caption="❓ 업로드한 문제 이미지", use_container_width=True)

        # 🔸 과목 정보 출력
        st.markdown(f"🧾 선택한 과목: **{subject}**")

        # 🔸 파이프라인 실행
        with st.spinner("🧠 문제를 분석하고 있습니다..."):

            ### 문학
            if subject == "문학":
                result = pipeline_literature(
                    question_image_path=question_img_path,
                    passage_image_path=passage_img_path,
                    retriever_answer=retriever_literature_answer,
                    retriever_recommend=retriever_literature_recommend,
                    tag_dict=tag_dict_literature,
                    show_images=False,
                    recommend_top_k=top_k
                )
                st.markdown("### 🧠 정답 및 해설")
                st.markdown(result["response"])

                st.markdown("### 🔁 유사 기출문제 추천")
                for problem in result["similar_problems"]:
                    st.markdown(f"#### 📌 유사 문제 {problem['index']}: {problem['question_code']}")

                    if os.path.exists(problem["passage_img"]):
                        st.image(problem["passage_img"], caption="📖 지문 이미지", use_container_width=True)
                    else:
                        st.markdown(f"❌ 지문 이미지 없음: `{problem['passage_img']}`")

                    if os.path.exists(problem["problem_img"]):
                        st.image(problem["problem_img"], caption="❓ 문제 이미지", use_container_width=True)
                    else:
                        st.markdown(f"❌ 문제 이미지 없음: `{problem['problem_img']}`")


            ### 비문학 (지문끼리 유사도 계산 -> 지문 1개, 문제 여러개)
            if subject == "비문학":
                response, similar_problems = pipeline_non_literature(
                    passage_img_path,
                    question_img_path,
                    top_k=top_k
                )

                st.markdown("### 🧠 정답 및 해설")
                st.markdown(response)

                st.markdown("### 🔁 유사 기출문제 추천")
                for i, problem in enumerate(similar_problems, 1):
                    st.markdown(f"#### 📌 유사 문제 {i}: {problem['year']}학년도 {problem['month']}월 p{problem['pNum']}")

                    if os.path.exists(problem["passage_img"]):
                        st.image(problem["passage_img"], caption="📖 지문 이미지", use_container_width=True)

                    for qnum, img_path in problem.get("problem_imgs", []):
                        if os.path.exists(img_path):
                            st.image(img_path, caption=f"❓ 문제 {qnum}번", use_container_width=True)


            ### 화법과 작문
            if subject == "화법과 작문":
                result = analyze_problem(
                    context_image_path=passage_img_path,
                    question_image_path=question_img_path,
                    top_k=top_k
                )

                st.markdown("### 🧠 정답 및 해설")
                st.markdown(result["gpt_response"])

                st.markdown("### 🔁 유사 기출문제 추천")

                for prob in result["similar_problems"]:
                    st.markdown(f"#### 📌 유사 문제: {prob['year']}년 {prob['month']}월 (문제 {prob['problem_number']}번)")

                    # 지문 이미지
                    if os.path.exists(prob["passage_img_path"]):
                        st.image(prob["passage_img_path"], caption="📖 지문 이미지", use_container_width=True)
                    else:
                        st.warning(f"❌ 지문 이미지 없음: {prob['passage_img_path']}")

                    # 문제 이미지
                    if os.path.exists(prob["problem_img_path"]):
                        st.image(prob["problem_img_path"], caption="❓ 문제 이미지", use_container_width=True)
                    else:
                        st.warning(f"❌ 문제 이미지 없음: {prob['problem_img_path']}")

                    st.markdown(f"**정답:** {prob.get('answer', '없음')}")
                    st.markdown(f"**해설:** {prob.get('explanation', '없음')}")



            ### 언어와 매체
            if subject == "언어와 매체":

        st.success("✅ 완료!")

Overwriting app.py


In [66]:
!npm install localtunnel

⠙⠹⠸⠼
up to date, audited 23 packages in 919ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼

In [68]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.238.182.40
⠙⠹⠸your url is: https://five-friends-throw.loca.lt
